In [1]:
import sqlite3
import random
from datetime import date, timedelta

# ----------------------------
# Config
# ----------------------------
DB_PATH = r"../data/appointments.db"

# January 2026 range
START_DATE = date(2026, 1, 1)
END_DATE   = date(2026, 1, 31)  # inclusive

# Demo services (edit freely)
SERVICES = [
    "Practice Growth Consultation",
    "RCM / Billing & Denials Review",
    "Benefits Verification & Authorization Setup",
    "Patient Engagement & Campaign Planning",
]

# Each service gets exactly one slot per day.
# You can set a fixed time per service to keep things simple and predictable.
SERVICE_TIME_MAP = {
    "Practice Growth Consultation": "10:00",
    "RCM / Billing & Denials Review": "11:00",
    "Benefits Verification & Authorization Setup": "12:00",
    "Patient Engagement & Campaign Planning": "13:00",
}

BOOKED_RATIO = 0.20

# Use a fixed seed so your demo data is reproducible (optional)
RANDOM_SEED = 42


# ----------------------------
# Helpers
# ----------------------------
def daterange_inclusive(start: date, end: date):
    """Yield dates from start to end (inclusive)."""
    cur = start
    while cur <= end:
        yield cur
        cur += timedelta(days=1)


def dummy_customer(i: int):
    """Generate simple fake customer data."""
    name = f"Demo Customer {i:03d}"
    phone = f"+2010{random.randint(10000000, 99999999)}"
    return name, phone


# ----------------------------
# Main seeding logic
# ----------------------------
def seed():
    random.seed(RANDOM_SEED)

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # 1) Create table (matches your simple schema, plus UNIQUE constraint)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS appointments (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      service TEXT NOT NULL,
      date TEXT NOT NULL,      -- YYYY-MM-DD
      time TEXT NOT NULL,      -- HH:MM (24h)
      status TEXT NOT NULL CHECK(status IN ('free','booked')),
      customer_name TEXT,
      phone TEXT,
      created_at TEXT NOT NULL DEFAULT (datetime('now')),
      UNIQUE(service, date, time)
    );
    """)

    # 2) Optional: clear only January 2026 data (so you can re-run safely)
    cur.execute("""
    DELETE FROM appointments
    WHERE date >= '2026-01-01' AND date <= '2026-01-31';
    """)

    # 3) Insert one slot per service per day
    total = 0
    booked_count = 0
    cust_counter = 1

    for d in daterange_inclusive(START_DATE, END_DATE):
        d_str = d.isoformat()

        for service in SERVICES:
            time_str = SERVICE_TIME_MAP.get(service, "10:00")

            is_booked = (random.random() < BOOKED_RATIO)
            status = "booked" if is_booked else "free"

            customer_name = None
            phone = None

            if is_booked:
                customer_name, phone = dummy_customer(cust_counter)
                cust_counter += 1
                booked_count += 1

            cur.execute("""
            INSERT OR IGNORE INTO appointments (service, date, time, status, customer_name, phone)
            VALUES (?, ?, ?, ?, ?, ?);
            """, (service, d_str, time_str, status, customer_name, phone))

            total += 1

    conn.commit()
    conn.close()

    print("Done seeding.")
    print(f"DB: {DB_PATH}")
    print(f"Total slots inserted (attempted): {total}")
    print(f"Booked slots (~20%): {booked_count}")
    print(f"Free slots: {total - booked_count}")
    print("Duration is assumed to be 30 minutes in app logic (fixed).")


if __name__ == "__main__":
    seed()


Done seeding.
DB: ../data/appointments.db
Total slots inserted (attempted): 124
Booked slots (~20%): 27
Free slots: 97
Duration is assumed to be 30 minutes in app logic (fixed).
